In [31]:
def load_documents(url: str):
    import pickle
    with open("../app/data/documents/" + str(url) + ".pkl", "rb") as f:
            return pickle.load(f)

documents = load_documents("27")

In [19]:
%pip install ipywidgets widgetsnbextension pandas-profiling llama-index-llms-ollama rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
from llama_index.llms.ollama import Ollama
from ragas.llms.base import LlamaIndexLLMWrapper

llm_ollama = Ollama(model="deepseek-r1:32b", request_timeout=120.0)

llm = LlamaIndexLLMWrapper(llm_ollama)

In [33]:
from ragas.embeddings.base import LangchainEmbeddingsWrapper
from langchain_ollama import OllamaEmbeddings

embedding_model = LangchainEmbeddingsWrapper(OllamaEmbeddings(
    model="nomic-embed-text:latest",

))

In [34]:
from ragas.testset.transforms import Transforms, apply_transforms, default_transforms

transforms = default_transforms(
                documents=list(documents),
                llm=llm,
                embedding_model=embedding_model
            )

In [35]:
from ragas.testset.graph import KnowledgeGraph, Node, NodeType

nodes = []
for doc in documents:
    node = Node(
        type=NodeType.DOCUMENT,
        properties={
            "page_content": doc.page_content,
            "document_metadata": doc.metadata,
        },
    )
    nodes.append(node)

kg = KnowledgeGraph(nodes=nodes)

In [36]:
from ragas.testset.transforms import apply_transforms

apply_transforms(kg, transforms)

Applying CustomNodeFilter:   0%|          | 0/123 [00:00<?, ?it/s]              Node 13e8c18e-514f-445c-a41a-7259623324ce does not have a summary. Skipping filtering.
Node c2ee0c5a-0768-406c-9021-d3832372e1a8 does not have a summary. Skipping filtering.
Node c1254093-467c-42c3-98d1-5d848f2313c4 does not have a summary. Skipping filtering.
Node 2a1936ec-bfea-408d-9144-a55aa37a6ffc does not have a summary. Skipping filtering.
Node 939caa2e-cead-486c-b742-85b8a261087d does not have a summary. Skipping filtering.
Node f63e3a46-e562-4215-8d4d-1ab1f26afa2d does not have a summary. Skipping filtering.
Node 16f9d355-38db-4186-a7b8-475faf5df48e does not have a summary. Skipping filtering.
Node a268ca72-beb2-4c64-aea5-1c834d30788b does not have a summary. Skipping filtering.
Node 2134772b-51ba-48f4-b766-132970879a3a does not have a summary. Skipping filtering.
Node 1fb6f126-71a7-4aea-9d21-3286e72ee7f2 does not have a summary. Skipping filtering.
Node 04bee285-4a49-4fd8-8321-eec9e420a2ef does not

In [37]:
kg.save("kg-27-32b.json")

In [38]:
from ragas.testset.persona import generate_personas_from_kg

personas = generate_personas_from_kg(kg=kg, llm=llm, num_personas=5)

Generating personas: 100%|██████████| 5/5 [01:51<00:00, 22.25s/it]


In [39]:
personas

[Persona(name='Data Protection Officer', role_description="Responsible for informing visitors about how their personal data is collected and processed on the school's website or services, ensuring technical and organizational measures are in place to protect user data."),
 Persona(name='Web Analytics Manager', role_description='Monitors and analyzes website traffic to improve user experience while ensuring compliance with privacy regulations.'),
 Persona(name='Data Privacy Officer', role_description='Ensures compliance with GDPR regulations regarding cookies and manages user consent for data collection.'),
 Persona(name='Data Protection Officer', role_description='Ensures compliance with GDPR regulations for data processing activities.'),
 Persona(name='Data Compliance Officer', role_description='Oversees compliance with contractual obligations for data sharing, ensuring proper monitoring of third-party transfers.')]

In [40]:
personas[0].model_dump_json()

persona_json = []
for persona in personas:
    persona_json.append(persona.model_dump_json())

persona_json

['{"name":"Data Protection Officer","role_description":"Responsible for informing visitors about how their personal data is collected and processed on the school\'s website or services, ensuring technical and organizational measures are in place to protect user data."}',
 '{"name":"Web Analytics Manager","role_description":"Monitors and analyzes website traffic to improve user experience while ensuring compliance with privacy regulations."}',
 '{"name":"Data Privacy Officer","role_description":"Ensures compliance with GDPR regulations regarding cookies and manages user consent for data collection."}',
 '{"name":"Data Protection Officer","role_description":"Ensures compliance with GDPR regulations for data processing activities."}',
 '{"name":"Data Compliance Officer","role_description":"Oversees compliance with contractual obligations for data sharing, ensuring proper monitoring of third-party transfers."}']

In [41]:
import json

with open("personas-32b.json", "w") as f:
    json.dump(persona_json, f)
